In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Basic Libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from warnings import filterwarnings
from collections import Counter

# Visualizations Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.offline as pyo
import plotly.express as px
import plotly.graph_objs as go
pyo.init_notebook_mode()
import plotly.figure_factory as ff
import missingno as msno

# Data Pre-processing Libraries
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split

# Modelling Libraries
from sklearn.linear_model import LogisticRegression,RidgeClassifier,SGDClassifier,PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC,LinearSVC,NuSVC
from sklearn.neighbors import KNeighborsClassifier,NearestCentroid
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.ensemble import VotingClassifier

# Evaluation & CV Libraries
from sklearn.metrics import precision_score,accuracy_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV,RepeatedStratifiedKFold


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<div style="background-color:#B4DBE9; color:#636363;">
    <h1><center>Introduction</center></h1>
</div>

This notebook is a condensed version of the LORETO COASTKEEPER® program's efforts addressing seawater quality monitoring in the most popular beach recreation areas of the Bahia de Loreto National Park from 2014 to 2019. Monitoring was conducted out in accordance with the Mexican Ministry of Health and the Federal Commission for the Protection against Health Risks' "Guidelines for Determining the Quality of Seawater for Recreational Use with Primary Contact" (**COFEPRIS**). The aquatic activity in which the body comes into direct touch with water and is ingested by the bather is known as primary contact.

## Description of Headings

Ver las descripciones en el dataset [AQUI](https://www.kaggle.com/hugoquintero/seawater-quality-pnbl-20142019?select=Master_Dataset_LCK_2014-2020.csv)
* **id:** Unique site ID for sampling processes
* **name:** Known name of sampling site
* **type:** recreativo means recreative site, pesquero means fishing ground site, Control means is a potential contamination source risk site
* **latitude:** Latitude site coordinates
* **longitude:** Longitude site coordinates
* date:
* hour:
* timestamp
* temperature:
* salinity:
* ph:
* dis_oxy:
* enterococcus:
* status:

<div style="background-color:#B4DBE9; color:#636363;">
    <h1><center>Importing The Dataset</center></h1>
</div>

In [ ]:
df=pd.read_csv('../input/seawater-quality-pnbl-20142019/Master_Dataset_LCK_2014-2020.csv')

Read the Headers

In [ ]:
df.info()

Output 5 rows of data

In [ ]:
df.head(20)

<div style="background-color:#B4DBE9; color:#636363;">
    <h1><center>Visualizations</center></h1>
</div>

Define Color palettes

In [ ]:
colors_blue = ["#132C33", "#264D58", '#17869E', '#51C4D3', '#B4DBE9']
colors_dark = ["#1F1F1F", "#313131", '#636363', '#AEAEAE', '#DADADA']
colors_green = ['#01411C','#4B6F44','#4F7942','#74C365','#D0F0C0']
sns.palplot(colors_blue)
sns.palplot(colors_green)
sns.palplot(colors_dark)

How many site samples are suitable
Status key:
1 = Suitable or Apt
0 = Not Suitable or Not Apt


In [ ]:
d= pd.DataFrame(df['status'].value_counts())
fig = px.pie(d,values='status',names=['Apt','Not Apt'],hole=0.4,opacity=0.6,
            color_discrete_sequence=[colors_green[3],colors_blue[3]],
             labels={'label':'Suitable','Suitable':'No. Of Samples'})

fig.add_annotation(text='status',
                   x=0.5,y=0.5,showarrow=False,font_size=14,opacity=0.7,font_family='monospace')

fig.update_layout(
    font_family='monospace',
    title=dict(text='Q. How many samples of water are Suitable?',x=0.47,y=0.98,
               font=dict(color=colors_dark[2],size=20)),
    legend=dict(x=0.37,y=-0.05,orientation='h',traceorder='reversed'),
    hoverlabel=dict(bgcolor='white'))

fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()

**Distribution of measured seawater parameters**

## pH Level
The pH of water is a measure of the acid–base equilibrium and, in most natural waters, is controlled by the carbon dioxide–bicarbonate–carbonate equilibrium system. An increased carbon dioxide concentration will therefore lower pH, whereas a decrease will cause it to rise. Temperature will also affect the equilibria and the pH. In pure water, a decrease in pH of about 0.45 occurs as the temperature is raised by 25 °C. 

In water with a buffering capacity imparted by bicarbonate, carbonate and hydroxyl ions, this temperature effect is modified (APHA, 1989). The pH of most drinking-water lies within the range 6.5–8.5. Natural waters can be of lower pH, as a result of, for example, acid rain or higher pH in limestone areas. Importance of pH The pH of water determines the solubility (amount that can be dissolved in the water) and biological availability (amount that can be utilized by aquatic life) of chemical constituents such as nutrients (phosphorus, nitrogen, and carbon) and heavy metals (lead, copper, cadmium, etc.). For example, in addition to affecting how much and what form of phosphorus is most abundant in the water, pH also determines whether aquatic life can use it. In the case of heavy metals, the degree to which they are soluble determines their toxicity. Metals tend to be more toxic at lower pH because they are more soluble. (Source: A Citizen's Guide to Understanding and Monitoring Lakes and Streams)



In [ ]:
# Configuracion de base del histograma
fig = px.histogram(df, x="ph", nbins=70, color='status', template='plotly_white', 
                  marginal='box', opacity=0.7, color_discrete_sequence=[colors_green[3],colors_blue[3]],
                  histfunc='count')

# linea en area pH neutro valor de 7
fig.add_vline(x=7, line_width=1, line_color=colors_dark[1],line_dash='dot',opacity=0.7)

# ajuste para centrar histograma e indicar cuando los valores son acidicos y basicos
fig.add_annotation(text='<7 is Acidic',x=5.5,y=70,showarrow=False,font_size=12)
fig.add_annotation(text='>7 is Basic',x=10,y=70,showarrow=False,font_size=12)
fig.add_annotation(text='1 = Apt',x=10,y=300,showarrow=False,font_size=11)
fig.add_annotation(text='0 = Not Apt',x=10,y=280,showarrow=False,font_size=11)

# Titulos y etiquetas en 
fig.update_layout(
    font_family='monospace',
    title=dict(text='pH Level Distribution at Bahia de Loreto National Park',x=0.5,y=0.95,
               font=dict(color=colors_dark[2],size=20)),
    xaxis_title_text='pH Level',
    yaxis_title_text='Count',
    legend=dict(x=1,y=0.96,bordercolor=colors_dark[4],borderwidth=0,tracegroupgap=5),
    bargap=0.3,
)

fig.show()

## Oxigeno Disuleto (OD)
El oxígeno disuelto es necesario para muchas formas de vida, incluidos peces, invertebrados, bacterias y plantas. La cantidad de oxígeno disuelto que se necesita varía de una criatura a otra. Los que se alimentan del fondo, los cangrejos, las ostras y los gusanos necesitan cantidades mínimas de oxígeno (1-6 mg / L), mientras que los peces de aguas poco profundas necesitan niveles más altos (4-15 mg / L). Los peces y organismos de agua salada tienen una mayor tolerancia a las bajas concentraciones de oxígeno disuelto, ya que el agua salada tiene una saturación de aire del 100% más baja que el agua dulce. Los requisitos de oxígeno disuelto de los peces de mar abierto y de aguas profundas son un poco más difíciles de rastrear, pero se han realizado algunos estudios en el área. Los crustáceos como los cangrejos y las langostas son organismos bentónicos (que viven en el fondo), pero aún requieren niveles mínimos de oxígeno disuelto. Dependiendo de la especie, los requisitos mínimos de OD pueden oscilar entre 4 mg / L y 1 mg / L. A pesar de ser habitantes del fondo, los mejillones, las ostras y las almejas también requieren un mínimo de 1-2 mg / L de oxígeno disuelto 29, por lo que se encuentran en aguas costeras menos profundas que reciben oxígeno de la atmósfera y fuentes fotosintéticas.

![Requisitos mínimos de oxígeno disuelto de los peces de agua salada](https://www.fondriest.com/environmental-measurements/wp-content/uploads/2013/11/dissolvedoxygen_levels-salt.jpg) 
![Los niveles bajos de oxígeno disuelto impactan significativamente a los organismos y comunidades marinas.](https://www.vims.edu/research/topics/_images/do_levels_big.jpg)


### Consecuencias de niveles inusuales de OD
Si las concentraciones de oxígeno disuelto caen por debajo de cierto nivel, las tasas de mortalidad de los peces aumentarán; Los peces costeros comienzan a evitar áreas donde el OD está por debajo de 3,7 mg / L, y especies específicas abandonan un área por completo cuando los niveles caen por debajo de 3,5 mg / L. Por debajo de 2.0 mg / L, los invertebrados también salen y por debajo de 1 mg / L incluso los organismos bentónicos muestran tasas de crecimiento y supervivencia reducidas.

La concentración de oxígeno disuelto en el agua del océano suele estar entre 7 y 8 miligramos por litro (mg / L). Si las concentraciones caen por debajo de 4 mg / L, los organismos comenzarán a reaccionar, con formas móviles evitando o migrando fuera del área. Las aguas con menos de 0,2 mg / L de oxígeno disuelto se denominan anóxicas y no pueden sustentar la mayoría de las formas de vida. Las aguas sin oxígeno disuelto mensurable se denominan hipóxicas.



In [ ]:
# Configuracion de base del histograma
fig = px.histogram(df, x="dis_oxy", nbins=50, color='status', template='plotly_white', 
                  marginal='box', opacity=0.7, color_discrete_sequence=[colors_green[3],colors_blue[3]],
                  histfunc='count')
                                   

# Reqerimientos oxigeno por especie
fig.add_annotation(text='menor 3.7<br> vida marina<br> abandonan area',x=3.3,y=500,showarrow=False,font_size=12)
fig.add_vline(x=3.8, line_width=1, line_color=colors_dark[1],line_dash='dot',opacity=0.7)
fig.add_annotation(text='la vida marina<br> evita este rango',x=4.5,y=500,showarrow=False,font_size=12)
fig.add_vline(x=5.5, line_width=1, line_color=colors_dark[1],line_dash='dot',opacity=0.7)
fig.add_annotation(text='este rango es<br> adecuado para la vida marina',x=6.9,y=500,showarrow=False,font_size=12)
fig.add_vline(x=8, line_width=1, line_color=colors_dark[1],line_dash='dot',opacity=0.7)
fig.add_annotation(text='rango de<br> saturacion al 100%',x=8.9,y=500,showarrow=False,font_size=12)

fig.add_annotation(text='1 = Apt',x=10,y=880,showarrow=False,font_size=11)
fig.add_annotation(text='0 = Not Apt',x=10,y=840,showarrow=False,font_size=11)



# Titulos y etiquetas en 
fig.update_layout(
    font_family='monospace',
    title=dict(text='Dissolved Oxygen Distribution at the Bahia de Loreto National Park',x=0.5,y=0.95,
               font=dict(color=colors_dark[2],size=20)),
    xaxis_title_text='DO Level in mg/L',
    yaxis_title_text='Count',
    legend=dict(x=1,y=0.96,bordercolor=colors_dark[4],borderwidth=0,tracegroupgap=5),
    bargap=0.3,
)

fig.show()

## Salinidad
La salinidad del agua viene dada por la cantidad de sales minerales disueltas en ella. Se ha aceptado comúnmente que la salinidad promedio del agua de mar ronda los 35 gramos por litro. 

In [ ]:
# Configuracion de base del histograma
fig = px.histogram(df, x="salinity", nbins=50, color='status', template='plotly_white', 
                  marginal='box', opacity=0.7, color_discrete_sequence=[colors_green[3],colors_blue[3]],
                  histfunc='count')

# linea valor medio
fig.add_vline(x=35, line_width=1, line_color=colors_dark[1],line_dash='dot',opacity=0.7)

# ajuste para centrar histograma e indicar cuando los valores son aptos y no aptos
fig.add_annotation(text='1 = Apt',x=38,y=1000,showarrow=False,font_size=11)
fig.add_annotation(text='0 = Not Apt',x=38,y=900,showarrow=False,font_size=11)

# Titulos y etiquetas en 
fig.update_layout(
    font_family='monospace',
    title=dict(text='Salinity Level Distribution at Bahia de Loreto National Park',x=0.5,y=0.95,
               font=dict(color=colors_dark[2],size=20)),
    xaxis_title_text='Salinity Level in ppt',
    yaxis_title_text='Count',
    legend=dict(x=1,y=0.96,bordercolor=colors_dark[4],borderwidth=0,tracegroupgap=5),
    bargap=0.3,
)

fig.show()

## Temperatura Superficial
Comparado con el aire, el agua del mar tiene capacidad calórica alta. Esto quiere decir, que se necesita más energía y más tiempo para cambiar o aumentar la temperatura del agua, en comparación a la temperatura del aire. De igual forma, una vez que el océano se calienta, se necesita mucho tiempo para que el agua pueda liberar o perder completamente ese calor. La mayor parte de la radiación solar es absorbida en los primeros 100 m de la columna de agua. Sin embargo, la temperatura superficial del mar se refiere a la temperatura del agua más cerca de la superficie del mar o del océano, la cual se obtiene en los primeros 20 m de la columna de agua. En nuestro caso 1 metro de la superficie.

Segun este paper ["Variación temporal y espacial de temperatura superficial del mar, clorofila a y productividad primaria en el golfo de California"](http://www.scielo.org.mx/scielo.php?script=sci_arttext&pid=S0185-38802013000200007) La temperatura promedio superficial de las aguas del Golfo de California fluctúa asi: El promedio anual de TSM fue más alto en la boca (25.57 °C), intermedio en la región central (24.63 °C) y más bajo en la RIG (23.24 °C). Los promedios anuales de TSM, Chl y PP para todo el golfo fueron 24.48 °C, 0.91 mg m-3 y 1.52 g C m-2 d-1, respectivamente. 



In [ ]:
# Configuracion de base del histograma
fig = px.histogram(df, x="temperature", nbins=100, color='status', template='plotly_white', 
                  marginal='box', opacity=0.7, color_discrete_sequence=[colors_green[3],colors_blue[3]],
                  histfunc='count')

# linea valor medio
fig.add_vline(x=25.57, line_width=1, line_color=colors_dark[1],line_dash='dot',opacity=0.7)
fig.add_vline(x=24.63, line_width=1, line_color=colors_dark[1],line_dash='dot',opacity=0.7)
fig.add_vline(x=23.23, line_width=1, line_color=colors_dark[1],line_dash='dot',opacity=0.7)
fig.add_vline(x=24.48, line_width=2, line_color=colors_dark[1],line_dash='dot',opacity=0.7)

# ajuste para centrar histograma e indicar cuando los valores son aptos y no aptos
fig.add_annotation(text='1 = Apt',x=34,y=200,showarrow=False,font_size=11)
fig.add_annotation(text='0 = Not Apt',x=34,y=190,showarrow=False,font_size=11)

# Titulos y etiquetas en 
fig.update_layout(
    font_family='monospace',
    title=dict(text='Salinity Level Distribution at Bahia de Loreto National Park',x=0.5,y=0.95,
               font=dict(color=colors_dark[2],size=20)),
    xaxis_title_text='Salinity Level in ppt',
    yaxis_title_text='Count',
    legend=dict(x=1,y=0.96,bordercolor=colors_dark[4],borderwidth=0,tracegroupgap=5),
    bargap=0.3,
)

fig.show()

## Enterococos
### Metas

1. Verificar la cantidad de Enterococcus a lo largo de los años y analizar si ha mejorado o no;
2. Comprobar si las vacaciones o feriados tienen alguna relación en la calidad del agua;
3. Trazar en un mapa las playas y la cantidad de Enterococcus recolectada en cada una de ellas.

### 1er Meta
- [ ] Verificar cantidad de Enterococcus a lo largo de los años y analizar si ha mejorado o no.
En primer lugar, revisemos la última fecha y la primera fecha para verificar si tenemos medidas de todo el año. 
**he detectado que el formato de la fecha no es el correcto debo corregir los datos para poder hacer esta meta ...**


### 3er Meta

- [ ] Trazar en un mapa las playas y la cantidad de Enterococcus recolectada en cada una de ellas.
El siguiente es un mapa de dispersión (heat map), usando Mapbox, donde se puede apreciar que cuanto más grande y oscuro es el círculo, más contaminada está el agua.

In [ ]:
fig3 = px.scatter_mapbox(df, lat="latitude", lon="longitude", hover_name="name", size='enterococcus', color='enterococcus', size_max= 50,
                        color_continuous_scale= px.colors.sequential.Inferno_r, zoom=9, height=500)
fig3.update_layout(mapbox_style="open-street-map")
fig3.update_layout(margin={"r":10,"t":10,"l":0,"b":0})
fig3.show()

Aqui otro tipo de mapa

In [ ]:
fig4 = px.scatter_mapbox(df, lat="latitude", lon="longitude", hover_name="name", size='enterococcus', color='enterococcus', size_max= 50,
                        color_continuous_scale= px.colors.sequential.solar_r, zoom=8, height=500)
fig4.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "source": [
             
                "https://stamen-tiles.a.ssl.fastly.net/watercolor/{z}/{x}/{y}.jpg"
                
            ]
        }
      ])
fig4.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig4.show()

## Matriz de gráficos de dispersión - Scatter Plot Matrix 
Esta herramienta ayuda encontrar la correlacion entre los diferentes todos los parametros medidos

**NOTA; falta eliminar los parametros no correlacionables**

In [ ]:
fig = px.scatter_matrix(df,df.drop('status',axis=1),height=1250,width=1250,template='plotly_white',opacity=0.7,
                        color_discrete_sequence=[colors_blue[3],colors_green[3]],color='status',
                       symbol='status',color_continuous_scale=[colors_green[3],colors_blue[3]])

fig.update_layout(font_family='monospace',font_size=10,
                  coloraxis_showscale=False,
                 legend=dict(x=0.02,y=1.07,bgcolor=colors_dark[4]),
                 title=dict(text='Matriz de gráficos de dispersión vs parametros',x=0.5,y=0.97,
                   font=dict(color=colors_dark[2],size=24)))
fig.show()

Ahora en tabla

In [ ]:
cor=df.drop('status',axis=1).corr()
cor

Un Mapa de calor para ver correlacion

In [ ]:
fig = px.imshow(cor,height=800,width=800,color_continuous_scale=colors_blue,template='plotly_white')

fig.update_layout(font_family='monospace',
                title=dict(text='Correlation Heatmap',x=0.5,y=0.93,
                             font=dict(color=colors_dark[2],size=24)),
                coloraxis_colorbar=dict(len=0.85,x=1.1) 
                 )

fig.show()

## Chequeo de datos faltantes  - missing data

In [ ]:
fig = msno.matrix(df,color=(0,0.5,0.5))

NOS FALTAN ALGUNOS DATOS DE TEMPERATURA Y SALINIDAD, Y MUCHOS DE  PH Y OD

In [ ]:
df.isnull().sum()

In [ ]:
df[df['status']==0].describe()

In [ ]:
df[df['status']==1].describe()

In [ ]:
df[df['status']==0][['temperature','salinity','ph','dis_oxy']].median()

In [ ]:
df[df['status']==1][['temperature','salinity','ph','dis_oxy']].median()

Podemos ver que la diferencia entre los valores medios y medianos de aptos y no aptos es mínima. Entonces usamos la mediana general de la característica para imputar los valores

In [ ]:
df['temperature'].fillna(value=df['temperature'].median(),inplace=True)
df['salinity'].fillna(value=df['salinity'].median(),inplace=True)
df['ph'].fillna(value=df['ph'].median(),inplace=True)
df['dis_oxy'].fillna(value=df['dis_oxy'].median(),inplace=True)

In [ ]:
df.isnull().sum()

## Estandarizando los datos
Tenemos un error del set de datos por manera que nombramos los id de sitio? SRS1?
para poder seguir con el analisis basado en Water Quality: Analysis (Plotly) and Modelling

In [ ]:
X = df.drop('status',axis=1).values
y = df['status'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
# WIP some errors to fix
#scaler = StandardScaler()
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)